<a href="https://colab.research.google.com/github/narayanvyas/ewaste-classification/blob/master/02_ewaste_detection_argumented.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install split-folders

In [ ]:
# Load the functions from helper.py

!wget https://raw.githubusercontent.com/narayanvyas/ml-helper/master/helper_functions.py

from helper_functions import pred_and_plot, create_tensorboard_callback,make_confusion_matrix, plot_loss_curves, getClassNames, walk_through_dir, compare_historys, unzip_data, unzip_data, calculate_results, removeData

--2021-12-12 03:04:33--  https://raw.githubusercontent.com/narayanvyas/ml-helper/master/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11081 (11K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.82K  --.-KB/s    in 0s      

2021-12-12 03:04:34 (25.2 MB/s) - ‘helper_functions.py’ saved [11081/11081]



In [ ]:
rootPath = '/content/drive/MyDrive/'
folderName = 'spares_six'


inputFolder = folderName
# outputFolder = folderName
outputFolder = folderName + '_splitted'

train_dir = outputFolder + '/train/'
test_dir = outputFolder + '/test/'

trainingRatio = 0.9
testRatio = 0.1
validationRatio = 0.0

batchSize = 32
epochs = 5
imgSize = (224, 224)

In [ ]:
unzip_data(rootPath + folderName + '.zip')
walk_through_dir(folderName)

In [ ]:
import splitfolders

splitfolders.ratio(inputFolder, output=outputFolder, seed=42, ratio=(trainingRatio, validationRatio, testRatio))

Copying files: 1800 files [00:00, 6259.61 files/s]


In [ ]:
import tensorflow as tf

# Load data in from directories and turn it into batches
train_data = tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                                                                            image_size=imgSize,
                                                                            label_mode="categorical",
                                                                            batch_size=batchSize)
test_data = tf.keras.preprocessing.image_dataset_from_directory(directory=test_dir,
                                                                           image_size=imgSize,
                                                                           label_mode="categorical")

Found 1619 files belonging to 6 classes.
Found 180 files belonging to 6 classes.


In [ ]:

# class_names = getClassNames(train_dir)
class_names = train_data.class_names
class_names
# class_names = getClassNames(train_dir)

['Fan', 'Hard Disk', 'Motherboard', 'Mouse', 'RAM', 'Speakers']

AttributeError: ignored

In [ ]:
# 1. Create base model with tf.keras.applications
base_model = tf.keras.applications.EfficientNetB0(include_top=False)

# 2. Freeze the base model (so the pre-learned patterns remain)
base_model.trainable = False

# 3. Create inputs into the base model
inputs = tf.keras.layers.Input(shape=(224, 224, 3), name="input_layer")

# 4. If using ResNet50V2, add this to speed up convergence, remove for EfficientNet
# x = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)(inputs)

# 5. Pass the inputs to the base_model (note: using tf.keras.applications, EfficientNet inputs don't have to be normalized)
x = base_model(inputs)
# Check data shape after passing it to base_model
print(f"Shape after base_model: {x.shape}")

# 6. Average pool the outputs of the base model (aggregate all the most important information, reduce number of computations)
x = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)
print(f"After GlobalAveragePooling2D(): {x.shape}")

# 7. Create the output activation layer
outputs = tf.keras.layers.Dense(len(class_names), activation="softmax", name="output_layer")(x)

# 8. Combine the inputs with the outputs into a model
model_0 = tf.keras.Model(inputs, outputs)

# 9. Compile the model
model_0.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

# 10. Fit the model (we use less steps for validation so it's faster)
history = model_0.fit(train_data,
                                 epochs=5,
                                 steps_per_epoch=len(train_data),
                                 validation_data=test_data,
                                 # Go through less of the validation data so epochs are faster (we want faster experiments!)
                                 validation_steps=int(len(test_data))
                                 # Track our model's training logs for visualization later
                                 )

KeyboardInterrupt: ignored

Exception ignored in: 'zmq.backend.cython.message.Frame.__dealloc__'
Traceback (most recent call last):
  File "zmq/backend/cython/checkrc.pxd", line 13, in zmq.backend.cython.checkrc._check_rc
KeyboardInterrupt


Shape after base_model: (None, 7, 7, 1280)
After GlobalAveragePooling2D(): (None, 1280)
Epoch 1/5
 7/51 [===>..........................] - ETA: 1:10 - loss: 1.7571 - accuracy: 0.2500

InvalidArgumentError: ignored